# UW-Madison GI Tract Image Segmentation



## Librerie

In [1]:
# IMPORTS
!pip install -q segmentation_models_pytorch

import segmentation_models_pytorch as smp
import pandas as pd
import os
from glob import glob
import torch
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import time
import datetime

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp


# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

## CONFIGURAZIONE 

In [2]:
DEBUG = True
SEED = 4321
BATCH_SIZE = 64
VAL_SIZE = 0.2
LEARNING_RATE = 2e-3
N_EPOCHS = 6
IMG_SIZE = (128,128)

## FUNZIONI UTILI

In [3]:
def seed_everything(seed=1464):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction


# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def show_img(img, mask=None):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    plt.imshow(img, cmap='bone')
    
    if mask is not None:
        plt.imshow(mask, alpha=0.5)
        handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
        labels = ["Large Bowel", "Small Bowel", "Stomach"]
        plt.legend(handles,labels)
    plt.axis('off')

def plot_loss(loss, val_loss):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.xticks(range(1,len(loss)+1))
    plt.plot(range(1,len(loss)+1), loss, label='train')
    plt.plot(range(1,len(val_loss)+1), val_loss, label='val')
    plt.title('loss')
    plt.legend()
    #plt.savefig('loss.png')
    return fig



In [4]:
seed_everything(SEED)

## CREAZIONE TRAINING SET

In [5]:
# Open the training dataframe and display the initial dataframe
DATA_DIR = "/kaggle/input/uw-madison-gi-tract-image-segmentation/"
TRAIN_DIR = "/kaggle/input/uw-madison-gi-tract-image-segmentation/train"

train_df = pd.read_csv(DATA_DIR + 'train.csv')

temp_df = pd.DataFrame({"id": train_df["id"][::3]})
temp_df["large_bowel"] = train_df["segmentation"][::3].values
temp_df["small_bowel"] = train_df["segmentation"][1::3].values
temp_df["stomach"] = train_df["segmentation"][2::3].values

train_df = temp_df

train_df["case"] = train_df["id"].apply(lambda x: int(x.split("_")[0].replace("case", "")))
train_df["day"] = train_df["id"].apply(lambda x: int(x.split("_")[1].replace("day", "")))
train_df["slice"] = train_df["id"].apply(lambda x: x.split("_")[3])




all_images = glob(os.path.join(DATA_DIR + 'train', "**", "*.png"), recursive=True)
train_df["path"] =all_images

train_df["width"] = train_df["path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[1]))
train_df["height"] = train_df["path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[2]))

train_df.reset_index(inplace=True, drop=True)
train_df.fillna('',inplace=True)

train_df

,id,large_bowel,small_bowel,stomach,case,day,slice,path,width,height
0,case123_day20_slice_0001,,,,123,20,0001,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
1,case123_day20_slice_0002,,,,123,20,0002,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
2,case123_day20_slice_0003,,,,123,20,0003,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
3,case123_day20_slice_0004,,,,123,20,0004,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
4,case123_day20_slice_0005,,,,123,20,0005,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,,,,30,0,0140,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
38492,case30_day0_slice_0141,,,,30,0,0141,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
38493,case30_day0_slice_0142,,,,30,0,0142,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
38494,case30_day0_slice_0143,,,,30,0,0143,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266


In [6]:
class GIDataset(torch.utils.data.Dataset):
    def __init__(self, df, subset="train"):
        self.df = df
        self.subset = subset

    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, index): 
        masks = np.zeros((IMG_SIZE[0], IMG_SIZE[1], 3), dtype=np.float32)
        img_path=self.df['path'].iloc[index]
        w=self.df['width'].iloc[index]
        h=self.df['height'].iloc[index]
        img = self.__load_img(img_path)
        if self.subset == 'train':
            for k,j in zip([0,1,2],["large_bowel","small_bowel","stomach"]):
                rles=self.df[j].iloc[index]
                mask = rle_decode(rles, shape=(h, w, 1))
                mask = cv2.resize(mask, IMG_SIZE)
                masks[:,:,k] = mask
        
        masks = masks.transpose(2, 0, 1)
        img = img.transpose(2, 0, 1)

        if self.subset == 'train': return torch.tensor(img), torch.tensor(masks)
        else: return torch.tensor(img)
        
    def __load_img(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        img = (img - img.min())/(img.max() - img.min())*255.0 
        img = cv2.resize(img, IMG_SIZE)
        img = np.tile(img[...,None], [1, 1, 3]) # gray to rgb
        img = img.astype(np.float32) /255.
        return img

In [7]:
val_df = train_df.sample(frac=VAL_SIZE)
train_df = train_df.drop(val_df.index).reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_dataset = GIDataset(train_df)
val_dataset = GIDataset(val_df)

# Creation of Pytorch DataLoaders with shuffle=True for the traing phase
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## MODELLO

In [8]:
model = smp.Unet(
        encoder_name='efficientnet-b1',      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.
        classes=3,        # model output channels (number of classes in your dataset)
        activation=None,
    )

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = smp.losses.DiceLoss(mode='multilabel')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b1-f1951068.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

In [9]:
torch.cuda.empty_cache()
history = {
    'train_loss' : [],
    'val_loss' : []
}
#----------TRAINING

# Measure the total training time for the whole run.
total_t0 = time.time()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Scaler for mixed precision
scaler = torch.cuda.amp.GradScaler()

# Setup for training with gpu
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
loss_fn.to(device)

# For each epoch...
for epoch_i in range(0, N_EPOCHS):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, N_EPOCHS))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode: Dropout layers are active
    model.train()

    pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc='Train ')
    for step, (images, masks) in pbar:         
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            loss   = loss_fn(y_pred, masks)

        # Perform a backward pass to compute the gradients in MIXED precision
        scaler.scale(loss).backward()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end.
        total_train_loss += loss.item()

        # Unscales the gradients of optimizer's assigned params in-place before the gradient clipping
        scaler.unscale_(optimizer)

        # Clip the norm of the gradients to 1.0.
        # This helps and prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient in MIXED precision
        scaler.step(optimizer)
        scaler.update()

    # Compute the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    history['train_loss'].append(avg_train_loss)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    t0 = time.time()

    # Put the model in evaluation mode: the dropout layers behave differently
    model.eval()

    total_val_loss = 0

    pbar = tqdm(enumerate(validation_dataloader), total=len(validation_dataloader), desc='Validation ')
    for step, (images, masks) in pbar:         
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with torch.no_grad():  
            y_pred = model(images)
            loss   = loss_fn(y_pred, masks)

        # Accumulate the validation loss.
        total_val_loss += loss.item()

    # Compute the average loss over all of the batches.
    avg_val_loss = total_val_loss / len(validation_dataloader)
    
    history['val_loss'].append(avg_val_loss)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

/opt/conda/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")



======== Epoch 1 / 6 ========
Training...


Train :   0%|          | 0/482 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Train :   0%|          | 1/482 [00:10<1:24:26, 10.53s/it]

...

Train :   1%|          | 4/482 [00:38<1:15:33,  9.48s/it]

Train :   1%|          | 5/482 [00:47<1:13:03,  9.19s/it]

Train :   1%|          | 6/482 [00:56<1:12:56,  9.19s/it]

Train :   1%|▏         | 7/482 [01:05<1:11:46,  9.07s/it]

Train :   2%|▏         | 8/482 [01:13<1:09:52,  8.84s/it]

In [ ]:
plot_loss(history['train_loss'], history['val_loss'])

In [ ]:
torch.save(model.cpu(), '/kaggle/working/trained_unet.pt')

In [ ]:
with torch.no_grad():
    pbar = tqdm(enumerate(validation_dataloader), total=len(validation_dataloader), desc='Test ')
    preds = []
    for step, (images, masks) in pbar:         
        images = images.to(device, dtype=torch.float)
        preds.append(model(images)) # .squeeze(0) # removing batch axis
        #preds   = nn.Sigmoid()(out) # removing channel axis
    preds = torch.cat(preds,dim=0).cpu().numpy()
    


## TEST SUBMISSION

In [ ]:
sub_df = pd.read_csv(DATA_DIR + 'sample_submission.csv')

test_df = pd.DataFrame({"id": sub_df["id"][::3]})
test_df["large_bowel"] = sub_df["segmentation"][::3].values
test_df["small_bowel"] = sub_df["segmentation"][1::3].values
test_df["stomach"] = sub_df["segmentation"][2::3].values

test_df["case"] = test_df["id"].apply(lambda x: int(x.split("_")[0].replace("case", "")))
test_df["day"] = test_df["id"].apply(lambda x: int(x.split("_")[1].replace("day", "")))
test_df["slice"] = test_df["id"].apply(lambda x: x.split("_")[3])
all_images = glob(os.path.join(DATA_DIR + 'test', "**", "*.png"), recursive=True)
test_df["path"] =all_images

test_df["width"] = test_df["path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[1]))
test_df["height"] = test_df["path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[2]))

test_df.reset_index(inplace=True, drop=True)
test_df.fillna('',inplace=True)

test_dataset = GIDatatset(test_df, subset='test')
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    

In [ ]:

t0 = time.time()

model.eval()
preds = []

pbar = tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc='Test ')
for step, (images, masks) in pbar:         
    images = images.to(device, dtype=torch.float)

    batch_size = images.size(0)

    with torch.no_grad():  
        y_pred = model(images)
        preds.append(y_pred.cpu().numpy())

print("  Test took: {:}".format(validation_time))

In [ ]:

masks = []
for i in range(len(preds)):
    for j in range(3):
        class_pred = preds[i,j,:,:]
        pred_img = cv2.resize(class_pred, (val_df['width'].iloc[i], val_df['height'].iloc[i]), interpolation=cv2.INTER_NEAREST) # resize probabilities to original shape
        pred_img = (pred_img>0.5).astype(dtype='uint8')    # classify
        masks.append(pred_img)
sub_df['prediction'] = pd.Series([rle_encode(m) for m in masks])
sub_df.to_csv("submission.csv", index=False)